In [2]:
import json
import pandas as pd
import sys
%load_ext autoreload
%autoreload 2
from llm_openai import safe_call_llm
from tqdm import tqdm
def read_multiline_json(file_path):
    json_data_list = []

    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line in lines:
        if line.strip():
            json_data = json.loads(line)
            json_data_list.append(json_data)

    return json_data_list


In [3]:
# table_file = '../dbgpt_hub/data/CSpider/tables.json'
# tables = json.load(open(table_file))
# # pd.DataFrame(tables)
# list(tables[0].keys())

In [4]:
# tables[0]['column_names_original'], tables[0]['table_names_original']

## chatgpt

In [5]:
# prompt = '帮我翻译成中文,只保留翻译内容,格式为list不变,不要省略,号:\n{}'
# x = tables[0]#['column_names_original']
# res = safe_call_llm(prompt.format(x['column_names_original']))
# # print(x)

In [6]:
tables = read_multiline_json('../dbgpt_hub/data/CSpider/gpt_tmp/tables_cn_gpt_new_revised.json')
len(tables)

166

In [20]:
import re
def contains_chinese(text):
    # 正则表达式匹配中文字符的范围
    chinese_regex = re.compile(r'[\u4e00-\u9fff]')
    return bool(chinese_regex.search(text))

def list_contains_no_chinese(lst):
    # 遍历列表中的每一个元素，检查是否包含中文字符
    for item in lst:
        if isinstance(item, str) and contains_chinese(item):
            return False
    return True


In [305]:
error_list = []
for j, item in enumerate(tables):
    table = item["table_names_original"]
    coloumns = item["column_names_original"][1:]
    coloumns_cn = item["cn_column_names_original"][1:]
    primary_key = item["primary_keys"]
    foreign_keys = item["foreign_keys"]
    # if item['db_id'] == 'department_management':
    #     break
    for i, name in enumerate(table):
        data = [coloumn[1] for coloumn in coloumns if coloumn[0] == i]
        data_cn = [coloumn[1] for coloumn in coloumns_cn if int(coloumn[0]) == i]
        if (len(data) != len(data_cn)) | (list_contains_no_chinese(data_cn)): # "数据表和数据列的中文名称数量不一致"
            error_list.append([j, i])
            # aaa
k = error_list[0][0]
# tables[k]['cn_column_names_original'][:4], k, len(error_list)
# error_list[0]
error_list

[[146, 0]]

In [296]:
# data

In [297]:
for a,b in zip(data_cn,data):
    print(a,b)

国家ID CountryId
国家编号 CountryName
国家名称 Continent


In [302]:
len(data),len(data_cn)

(4, 4)

In [303]:
tables[k]['column_names_original']
# data_cn,data

[[-1, '*'],
 [0, 'f1'],
 [0, 'f2'],
 [1, 'id'],
 [1, 'uid'],
 [1, 'text'],
 [1, 'createdate'],
 [2, 'uid'],
 [2, 'name'],
 [2, 'email'],
 [2, 'partitionid'],
 [2, 'followers']]

In [304]:
tables[k]['cn_column_names_original']

[[-1, '星号'],
 [0, 'f1'],
 [0, 'f2'],
 [1, 'id'],
 [1, 'uid'],
 [1, '文本'],
 [1, '创建日期'],
 [2, 'uid'],
 [2, '名称'],
 [2, '电子邮件'],
 [2, '分区id'],
 [2, '关注者']]

In [306]:
# a = [[-1, '*'],
#  [0, 'ContId'],
#  [0, '大陆'],
#  [1, '国家编号'],
#  [1, '国家名称'],
#  [1, '大陆'],
#  [2, 'Id'],
#  [2, '制造商编号'],
#  [2, '全名'],
#  [2, '国家'],
#  [3, 'ModelId'],
#  [3, '制造商'],
#  [3, '型号'],
#  [4, 'MakeId'],
#  [4, '型号'],
#  [4, '制造商'],
#  [5, 'Id'],
#  [5, '每加仑英里数'],
#  [5, '气缸数'],
#  [5, '排量'],
#  [5, '马力'],
#  [5, '重量'],
#  [5, '加速'],
#  [5, '年份']]
# tables[k]['cn_column_names_original'] = a

In [307]:
with open('../dbgpt_hub/data/CSpider/gpt_tmp/tables_cn_gpt_new_revised.json', 'w') as f:
    f.write("\n".join([json.dumps(i, ensure_ascii=False) for i in tables]))

with open('../dbgpt_hub/data/CSpider/tables_cn.json', 'w') as f:
    json.dump(tables, f, ensure_ascii=False, indent=4)

In [118]:
=

In [116]:
error_list

[]

In [104]:
# for i,x in enumerate(data):
#     tmp = eval(x['cn_column_names_original'])
#     tmp = []
#     x['cn_column_names_original'] = tmp
#     x['cn_table_names_original'] = eval(x['cn_table_names_original'])


In [105]:
data[30]['cn_column_names_original']

[[-1, '*'],
 [0, '服务_ID'],
 [0, '服务类型_代码'],
 [1, '参与者_ID'],
 [1, '参与者类型_代码'],
 [1, '参与者详情'],
 [2, '事件_ID'],
 [2, '服务_ID'],
 [2, '事件详情'],
 [3, '事件_ID'],
 [3, '参与者_ID']]

In [135]:
INSTRUCTION_ONE_SHOT_PROMPT_CN = """\
我希望你充当示例数据库的SQL终端。你只需要返回 SQL 命令给我。首先，我会向你展示一些指令及其正确的 SQL 响应示例。然后，我会给你一个新指令，你需要编写一个适当完成请求的 SQL 响应。
### 示例1 Instruction:
该数据库包含名为 employee、salary 和 position 的表。\
表 employee 包含包含的数据列如下:
employee_id [员工ID]
name [姓名]
age [年龄]
position_id [职位ID]
employee_id 是主键。
表 salary 包含包含的数据列如下:
employee_id [员工ID]
amount [金额]
date [日期]
employee_id 是主键。
表 position 包含包含的数据列如下: 
position_id [职位ID]
title [标题]
department [部门]
position_id 是主键。
salary 表中的 employee_id 是 employee 表中 employee_id 的外键。\
employee 表中的 position_id 是 position 表中 position_id 的外键。\
\n### 示例1 Input:\n列出'Engineering'部门员工的姓名和年龄。\n\
\n### 示例1 Response:\nSELECT employee.name, employee.age FROM employee JOIN position ON employee.position_id = position.position_id WHERE position.department = 'Engineering';\
\n###新指令：\n{}\n"""

# print(INSTRUCTION_ONE_SHOT_PROMPT_CN)
len(INSTRUCTION_ONE_SHOT_PROMPT_CN)

755